In [1]:
#Package imports
import logging
import os
import pytz
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from financial_modelling.modelling.SVIModel import SVIModel as svi
from financial_modelling.data_acquisition.database_fetcher import DatabaseFetcher as dbf
from financial_modelling.data_pre_processing.IVPreprocessor import IVPreprocessor as ivp
from financial_modelling.data_acquisition.list_of_files_fetcher import ListOfFilesFetcher as loff
import pandas as pd
index_of_quote = 10

We need to get first the fitted dates from the folder where the fitted data is present

In [2]:
#List of fitted SVI parameters
folder = r"E:\OutputParamsFiles\OutputFiles"
loff = loff()
loff.fetch(folder)
loff.get_unixtimestamp()
list_of_fitted_dates = loff.list_of_dates
date = list_of_fitted_dates[index_of_quote]

In [3]:
#Function get_unixtimestamp_readable for logging purposes
def get_unixtimestamp_readable(unixtimestamp):
    logging.info(datetime.fromtimestamp(unixtimestamp, pytz.timezone("US/Eastern")).strftime('%d-%m-%Y %H:%M'))

Need to implement a function that give the corresponding data in the database.

I. The database connection string

In [4]:
#Connection configuration
DB_CONFIG = {
    'server': 'DESKTOP-DK79R4I',  # Your server name
    'database': 'DataMining',     # Your database name
}

#Define pyodbc-compatible connection string
connection_string = (
        f"DRIVER={{SQL Server}};"
        f"SERVER={DB_CONFIG['server']};"
        f"DATABASE={DB_CONFIG['database']};"
        f"Trusted_Connection=yes;"
)

#Define the query string
query = f"""
        SELECT TOP(6302) *
        FROM [DataMining].[dbo].[OptionData]
        WHERE [QUOTE_UNIXTIME] = '{date}'
        """
database_fetcher = dbf(connection_string)
raw_data = database_fetcher.fetch(query)
database_fetcher.close()

INFO:root:DatabaseFetcher initialized with DRIVER={SQL Server};SERVER=DESKTOP-DK79R4I;DATABASE=DataMining;Trusted_Connection=yes;


pyodbc connection established in 6151.79 ms


c:\users\paula\onedrive\documents\financial modelling package\financial_modelling\data_acquisition\database_fetcher.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.conn)


Query executed in 0.2472 seconds


AttributeError: 'DatabaseFetcher' object has no attribute 'close_connection'

Need to extract the needed data from raw_data

In [24]:
#Preprocessor preprocessing rawa data
processed_data = ivp(raw_data).preprocess()
expiry = processed_data["EXPIRE_UNIX"].unique()[0]
processed_data = processed_data[processed_data["EXPIRE_UNIX"] == expiry]
log_moneyness = np.array(processed_data["Log_Moneyness"])

In [25]:
#Get the fitted SVI parameters
def get_the_fitted_params_file(date, folder):
    file_path = os.path.join(folder, f"output_{date}.csv")
    fitted_params_file = pd.read_csv(file_path, sep = ",")
    return fitted_params_file
fitted_params_file = get_the_fitted_params_file(date, folder)
expriry_specific_params = fitted_params_file[fitted_params_file["EXPIRE_DATE"] == expiry]
params = {"a":expriry_specific_params["a"],
          "b":expriry_specific_params["b"],
          "rho":expriry_specific_params["rho"],
          "m":expriry_specific_params["m"],
          "sigma":expriry_specific_params["sigma"]}

svi_values = svi().svi(log_moneyness, params["a"], params["b"], params["rho"], params["m"], params["sigma"])

ValueError: Length of values (113) does not match length of index (1)